In [ ]:
# Imports
import pandas as pd
import numpy as np

df_coaches = pd.read_csv('dataset/processed/coaches_processed.csv')

stats_df_coaches = df_coaches.drop("stint", axis=1)

stats_df_coaches["Win Rate"] = np.where(
    (stats_df_coaches['post_wins'] == 0) & (stats_df_coaches['post_losses'] == 0),
    stats_df_coaches['won'] / (stats_df_coaches['won'] + stats_df_coaches['lost']),
    ((stats_df_coaches['won'] + stats_df_coaches['post_wins']) / (stats_df_coaches['won'] + stats_df_coaches['lost'] + stats_df_coaches['post_wins'] + stats_df_coaches['post_losses']))
)

#Estreante ou não
stats_df_coaches['Cumulative Win Rate'] = 0.0
def calculate_cumulative_win_rate(group):
    group.loc[group.index[0], 'Cumulative Win Rate'] = group.loc[group.index[0], 'Win Rate']
    for i in range(1, len(group)):
        group.loc[group.index[i], 'Cumulative Win Rate'] = (
            group.loc[group.index[i], 'Win Rate'] * 0.7 + group.loc[group.index[i-1], 'Cumulative Win Rate'] * 0.3
        )
    group['Cumulative Win Rate'] = group['Cumulative Win Rate'].shift(1)
    return group

# Apply the function to each group of coachID
stats_df_coaches = stats_df_coaches.groupby('coachID').apply(calculate_cumulative_win_rate).reset_index(drop=True)
stats_df_coaches['Rookie'] = stats_df_coaches['Cumulative Win Rate'].isna()
stats_df_coaches.drop(columns=['won', 'lost', 'post_wins', 'post_losses', 'Win Rate'], inplace=True) 

df_teams = pd.read_csv('dataset/processed/teams_test.csv')
stats_df_coaches= stats_df_coaches.drop_duplicates(subset=['tmID', 'year'], keep='first')
df_teams = pd.merge(df_teams, stats_df_coaches, on=['tmID', 'year'], how='left')

df_teams.to_csv('dataset/processed/teams_test.csv', index=False)
stats_df_coaches.head()
#stats_df_players.dropna(axis=0, how='any', inplace=True)
#stats_df_players.isna().sum()

#gajos com 0 minutos
#df_players.loc[df_players['minutes'] <= 0, ['minutes', 'PostMinutes']]


C:\Users\Utilizador\AppData\Local\Temp\ipykernel_7420\4114766519.py:34: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  stats_df_coaches = stats_df_coaches.groupby('coachID').apply(calculate_cumulative_win_rate).reset_index(drop=True)


,coachID,year,tmID,Cumulative Win Rate,Rookie
0,adamsmi01w,5,WAS,NaN,True
1,adubari99w,1,NYL,NaN,True
2,adubari99w,2,NYL,0.615385,False
3,adubari99w,3,NYL,0.626721,False
4,adubari99w,4,NYL,0.573016,False
